In [4]:
import pandas as pd
file = 'filtered_w_missing_cbs_incl14.csv'
cbs_file_raw = pd.read_csv(file, low_memory=False)

In [5]:
import numpy as np
import pandas as pd
from scipy.stats import linregress
from sklearn.preprocessing import StandardScaler, LabelEncoder
from itertools import islice


In [12]:
def count_consecutive_mv(buurt_values: list):
    """Counts what the highest consecutive values missing"""

   
    start_index, end_index = None, None
    consecutive_count, highest_count = 0, 0
    temp_start = None  # Temporary start index for the current streak
    
    
    for i in range(len(buurt_values)):
        if pd.isna(buurt_values[i]) and pd.isna(buurt_values[i - 1]):
            consecutive_count += 1
            
            if consecutive_count > highest_count:
                highest_count = consecutive_count
        
        elif pd.isna(
            buurt_values[i]) and pd.notna(buurt_values[i - 1]):
            consecutive_count = 1

            temp_start = i
            
            if consecutive_count > highest_count:
                highest_count = consecutive_count
                start_index = temp_start

        else:
            
            if pd.isna(buurt_values[i - 1]):
                if consecutive_count == highest_count:
                    end_index = (i - 1)
                    start_index = temp_start

            consecutive_count = 0
                
            
    
    return highest_count, start_index, end_index

# Example usage
data1 = pd.Series([pd.NA, pd.NA, pd.NA, 308, 393, 421, pd.NA, 479, 598, 605])
data1 = pd.Series([pd.NA, pd.NA, 308, pd.NA, pd.NA, pd.NA, 421, pd.NA, 598, 605])

result = count_consecutive_mv(data1.tolist())
print(f"Longest streak: {result[0]}, Start index: {result[1]}, End index: {result[2]}")

Longest streak: 3, Start index: 3, End index: 5


## 1. Techniques for handling missing values

### 1.1 Inter- and extrapolation

In [13]:
import pandas as pd

def slope(y1, y2, x1, x2):
    return (y2 - y1) / (x2 - x1)

def linear_interpolation(buurt_data):
    
    buurt_values = buurt_data.astype('Float64')
    buurt_values = buurt_values.reset_index(drop=True)

    smal_known_i = buurt_values.first_valid_index()
    larg_known_i = buurt_values.last_valid_index()


    # Calculate overall slope using first and last known values
    line_slope = slope(
        buurt_values.iloc[smal_known_i], 
        buurt_values.iloc[larg_known_i], 
        smal_known_i, 
        larg_known_i
    )

    #print(f"Overall slope: {line_slope}")

    while buurt_values.isnull().sum() > 0:
        # Extrapolation: missing value at the beginning
        if pd.isna(buurt_values.iloc[0]) and pd.notna(buurt_values.iloc[-1]):
            # Distance from first known point
            dist = smal_known_i - 0
            total_dist = larg_known_i - smal_known_i
            weight = dist / total_dist  # Weight factor based on distance
            adjusted_slope = weight * line_slope  # Scale slope
            buurt_values.iloc[0] = buurt_values.iloc[1] - adjusted_slope

        # Extrapolation: missing value at the end
        elif pd.isna(buurt_values.iloc[-1]) and pd.notna(buurt_values.iloc[0]):
            # Distance from last known point
            dist = len(buurt_values) - 1 - larg_known_i
            total_dist = larg_known_i - smal_known_i
            weight = dist / total_dist  # Weight factor based on distance
            adjusted_slope = weight * line_slope  # Scale slope
            buurt_values.iloc[-1] = buurt_values.iloc[-2] + adjusted_slope

        # Extrapolation: missing values at both ends
        elif pd.isna(buurt_values.iloc[0]) and pd.isna(buurt_values.iloc[-1]):
            # Apply weighted slope at both ends
            dist1 = smal_known_i - 0
            dist2 = len(buurt_values) - 1 - larg_known_i
            total_dist = larg_known_i - smal_known_i
            weight1 = dist1 / total_dist
            weight2 = dist2 / total_dist
            adjusted_slope1 = weight1 * line_slope
            adjusted_slope2 = weight2 * line_slope
            buurt_values.iloc[0] = buurt_values.iloc[1] - adjusted_slope1
            buurt_values.iloc[-1] = buurt_values.iloc[-2] + adjusted_slope2

        # Otherwise, perform linear interpolation
        else:
            buurt_values = buurt_values.interpolate(method="linear", limit_direction="both")

    return buurt_values

# Example data
data1 = pd.Series([pd.NA, 2200, pd.NA, 2270, pd.NA, 2510, 2860, 3010, 3050, pd.NA])

# Apply function with weighted extrapolation
result = linear_interpolation(data1)
print(result)


0    2182.653061
1         2200.0
2         2235.0
3         2270.0
4         2390.0
5         2510.0
6         2860.0
7         3010.0
8         3050.0
9    3067.346939
dtype: Float64


In [14]:
def slope(y1, y2, x1, x2):
    return (y2 - y1) / (x2 - x1)



def consecutive_extrapolation(buurt_data, start_index, end_index):

    buurt_values = pd.Series(list(buurt_data))
    # Make sure right data type, improve 
    buurt_values = buurt_values.astype('Float64')

    smal_known_i = buurt_values.first_valid_index()
    larg_known_i = buurt_values.last_valid_index()
    
    #Compute slope between first and last known point
    line_slope = slope(buurt_values.iloc[smal_known_i], buurt_values.iloc[larg_known_i], smal_known_i, larg_known_i)
    
    # When the consecutive missing values occur at the beginning, begin iterating at the end.
    if start_index == 0:
        loop_start = len(buurt_values) - 1
        loop_end = start_index - 1
        direction = -1
    
    # When the consecutive missing values occur at the end or in between, begin iterating at the start.
    elif start_index != 0:
        loop_start = 0
        loop_end = len(buurt_values)
        direction = 1
        

    for i in range(loop_start, loop_end, direction):

        valid_indices = buurt_values[buurt_values.notna()].index

        if pd.isna(buurt_values.iloc[i]):  # Only extrapolate missing values
            
            prev_index = valid_indices[valid_indices < i]
            next_index = valid_indices[valid_indices > i]

            if not prev_index.empty and not next_index.empty: 
                
                prev_known_index = prev_index.max()
                prev_known_value = buurt_values.iloc[prev_known_index] 
                next_known_index = next_index.min()
                next_known_value = buurt_values.iloc[next_known_index]

                line_slope = slope(prev_known_value, next_known_value, prev_known_index, next_known_index)
                #print(slope)

                if start_index == 0:
                    buurt_values.iloc[i] = buurt_values.iloc[i + 1] - line_slope
                    #buurt_values.iloc[i] = next_known_value + slope * (i - next_known_index)

                else: 
                    buurt_values.iloc[i] = buurt_values.iloc[i - 1] + line_slope
                    #buurt_values.iloc[i] = prev_known_value + slope * (i - prev_known_index)

            
            elif prev_index.empty and not next_index.empty: 
                next_known_index = next_index.min()
                next_known_value = buurt_values.iloc[next_known_index]
                
                second_known_index = next_index[1]                
                second_known_value = buurt_values.iloc[second_known_index]

                line_slope = slope(next_known_value, second_known_value, next_known_index, second_known_index)
                
                buurt_values.iloc[i] = buurt_values.iloc[i+1] - line_slope


            elif not prev_index.empty and next_index.empty: 
                prev_known_value = buurt_values.iloc[prev_index.max()]
                
                second_known_index = prev_index[len(prev_index)-2]
                second_known_value = buurt_values.iloc[second_known_index]

                line_slope = slope(second_known_value, prev_known_value, second_known_index, prev_index.max())

                buurt_values.iloc[i] = buurt_values.iloc[i-1] + line_slope 
                

    return buurt_values


data1 = pd.Series([pd.NA, pd.NA, pd.NA, 2270, 2310, 2510, 2860, 3010, 3050, pd.NA])
result = consecutive_extrapolation(data1, 0, 2)
print(result)


0    2150.0
1    2190.0
2    2230.0
3    2270.0
4    2310.0
5    2510.0
6    2860.0
7    3010.0
8    3050.0
9    3090.0
dtype: Float64


### 1.2 Spline

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline

def spline_interpolation(buurt_data):

    buurt_values = buurt_data.astype('Float64')
    
    x = np.arange(len(buurt_values))
    
    # Create a mask for the valid (non-NaN) values
    mask = ~buurt_values.isna()
    
    # Fit cubic spline to the valid data points
    cs = CubicSpline(x[mask], buurt_values[mask], bc_type='natural', extrapolate=False)
    
    # Generate new x values for interpolation
    x_new = np.linspace(x.min(), x.max(), 100)
    y_new = cs(x_new)
    
    # Interpolate the missing values in the original data
    buurt_values[buurt_values.isna()] = cs(x[buurt_values.isna()])
    
    return buurt_values

# # Real values [299, 310, 315, 313, 217, 226, 233, 258, 316, 337]
# data1 = pd.Series([np.nan, np.nan, 315, np.nan, 217, np.nan, np.nan, 258, 316, 337])  
# imputed_values = spline_interpolation(data1)

# print(imputed_values)


In [11]:
import numpy as np
import pandas as pd

def variance_buurt(buurt_data, variance, non_missing_values):
    """
    Calculates variance for a neighborhood and imputes missing values:
    - If variance < 0.01, imputes with the mean.
    - Otherwise, uses forward fill (last seen value).


    Returns the imputed neighborhood data.
    """
    
    # Decide imputation strategy, low immpute mean
    if variance < 0.01:
        return buurt_data.fillna(non_missing_values.mean())  # Impute with mean
        
    else:
        return buurt_data.ffill()  # Fix: Use .ffill() directly


# data23 = pd.Series([np.nan, np.nan, 1, np.nan, 1, np.nan, np.nan, 1, 1, 1])  
# variance = np.var(data23.dropna(), ddof=1)  # Sample variance
# print(variance)

# variance_buurt(data23, variance, data23.dropna())
# # imputed_values = spline_interpolation(data1)



## Structure Handling MV's

In [16]:
### import matplotlib.pyplot as plt
from itertools import islice
from scipy.stats import linregress


def main_strcuture_handling_mv(df):
    """Fills missing values based on linear regression and interpolation techniques."""
    # Create a new DataFrame to store the results and copy the first 5 rows entirely from the original DataFrame
    new_df = cbs_file_raw[['regio', 'gm_naam', 'gwb_code', 'ind_wbi', 'year']].copy()   

    # Select the colomns
    columns = df.columns

    
    for col in columns[5:]:
        print(f'Processing column: {col}')
        
        missing_count = df[col].isnull().sum()
        if missing_count == 0:
            new_df[col] = df[col]
            continue

        # Groepeer buurten om er door te itereren en per colom de ontbrekende waarde te vinden
        buurten = df.groupby('gwb_code')[col]
        
        for buurt_code, buurt_data in buurten:
            row_index = buurt_data.index

            if buurt_data.isnull().any():
                x = buurt_data.dropna().index
                y = buurt_data.dropna()

                if (x.empty or y.empty) or (len(buurt_data[buurt_data.isnull()]) >= 7): 
                    new_df.loc[row_index, col] = buurt_data  # Leave it for MICE later
                
                else: #tef
                    # First check on variance, if low impute data with mean, of fill
                    variance = np.var(buurt_data.dropna(), ddof=1)  # Sample variance
                    if variance <= 1:
                        handled_data = variance_buurt(buurt_data, variance, buurt_data.dropna())

                    else: #tef
                        slope, intercept, r_value, _, _ = linregress(x, y)
                        r_sqrt = r_value**2 
    
                        # Wanneer er 1 missing value is in de buurt
                        if len(buurt_data[buurt_data.isnull()]) == 1: 
                            if r_sqrt >= 0.7:
                                #print(buurt_data)
                                handled_data = linear_interpolation(buurt_data) 
                                
                            # Als r-waarde kleiner of gelijk is aan 75%: Gebruik .... voor ontbrekende waarden
                            elif r_sqrt < 0.7:
                                handled_data = spline_interpolation(buurt_data)                          
                                #print(buurt_data)
                                #for original, handled in zip(buurt_data, handled_data):
                                #print(f"Case2: {str(original):<15} {str(handled):<15}")
           
                        
                        # There are multiple missing_values
                        else:
                            consecutive_coount, start_index, end_index = count_consecutive_mv(list(buurt_data))
        
                            # When there arent any consecutive missing values
                            if consecutive_coount == 1:
                                
                                if r_sqrt >= 0.7:
                                    handled_data = linear_interpolation(buurt_data)
    
                                elif r_sqrt < 0.7:
                                    handled_data = spline_interpolation(buurt_data)
                                    
                            elif consecutive_coount > 1:
                                if r_sqrt >= 0.7:
                                    handled_data = consecutive_extrapolation(buurt_data, start_index, end_index)
                                    
                                elif r_sqrt < 0.7:
                                    handled_data = spline_interpolation(buurt_data)  
                                    
                    
                # Make sure handled data has same dtype and index as buurt_data
                handled_data = handled_data.astype(buurt_data.dtype)
                handled_data.index = buurt_data.index 
                
                # Update the new DataFrame with handled data
                new_df.loc[row_index, col] = handled_data

            else: 
                # If no missing values, copy the original data
                new_df.loc[row_index, col] = buurt_data


    return new_df
                    


new_df = main_strcuture_handling_mv(cbs_file_raw)

Processing column: woz
Processing column: a_inw
Processing column: a_man
Processing column: a_vrouw
Processing column: a_geb
Processing column: p_geb
Processing column: a_ste
Processing column: p_ste
Processing column: a_hh
Processing column: g_hhgro
Processing column: bev_dich
Processing column: a_woning
Processing column: p_1gezw
Processing column: p_mgezw
Processing column: p_leegsw
Processing column: p_koopw
Processing column: p_huurw
Processing column: p_wcorpw
Processing column: p_ov_hw
Processing column: g_ele
Processing column: g_ele_vw
Processing column: g_ele_hu
Processing column: g_ele_ko
Processing column: g_gas
Processing column: g_gas_vw
Processing column: g_gas_hu
Processing column: g_gas_ko
Processing column: a_inkont
Processing column: g_ink_po
Processing column: g_ink_pi
Processing column: p_ink_li
Processing column: p_ink_hi
Processing column: p_hh_li
Processing column: p_hh_hi
Processing column: p_hh_lkk
Processing column: p_hh_osm
Processing column: a_soz_wb
Proces

In [17]:
before_mice_df = new_df.copy()

In [18]:
from sklearn.preprocessing import LabelEncoder 
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler


def convert_to_num(df):
    labelencoder = LabelEncoder() 
    
    # Fit and transform both 'gm_naam' and 'regio'
    df['gm_naam'] = labelencoder.fit_transform(df['gm_naam'])
    df['regio'] = labelencoder.fit_transform(df['regio'])

    return df  # Return the entire dataframe with transformed categorical columns

def df_for_training(df):
    # Drop unwanted columns
    df = df.drop(columns=['gwb_code', 'ind_wbi'], errors='ignore')

    # Convert non-numeric columns to numeric
    for col in df.columns:
        if df[col].dtype != 'float64' and df[col].dtype != 'int64':
            df[col] = pd.to_numeric(df[col], errors='coerce')  # Convert any other non-numeric columns

    return df

def mice(df):
    converted_df = df.copy()
    
    # Replace gemeentenamen ('gm_naam') and 'regio' with integers
    converted_df = convert_to_num(converted_df)
    
    # Prepare the DataFrame for training by handling numeric conversions
    df_train = df_for_training(converted_df)
    
    numeric_columns = df_train.select_dtypes(include=[np.number]).columns

    scaler = StandardScaler()
    scaled_train = scaler.fit_transform(df_train)

    # Impute the missing values
    imputer = IterativeImputer(random_state=100, max_iter=50, tol=1e-4, verbose=2)
    imputed_scaled = imputer.fit_transform(scaled_train)

    # Reverse scaling and update the DataFrame
    df_imputed = pd.DataFrame(scaler.inverse_transform(imputed_scaled), columns=numeric_columns)
    converted_df[numeric_columns] = df_imputed

    return df_imputed
                                    
non_mis_df = mice(before_mice_df)

[IterativeImputer] Completing matrix with shape (51854, 69)
[IterativeImputer] Ending imputation round 1/50, elapsed time 26.20
[IterativeImputer] Change: 62.510786464896356, scaled tolerance: 0.012165092489749533 
[IterativeImputer] Ending imputation round 2/50, elapsed time 51.70
[IterativeImputer] Change: 19.884420168915454, scaled tolerance: 0.012165092489749533 
[IterativeImputer] Ending imputation round 3/50, elapsed time 77.65
[IterativeImputer] Change: 6.954257156254338, scaled tolerance: 0.012165092489749533 
[IterativeImputer] Ending imputation round 4/50, elapsed time 103.36
[IterativeImputer] Change: 4.054598052139049, scaled tolerance: 0.012165092489749533 
[IterativeImputer] Ending imputation round 5/50, elapsed time 128.87
[IterativeImputer] Change: 2.979146507195815, scaled tolerance: 0.012165092489749533 
[IterativeImputer] Ending imputation round 6/50, elapsed time 153.78
[IterativeImputer] Change: 2.471853814166466, scaled tolerance: 0.012165092489749533 
[IterativeI

/home/wouter/.local/lib/python3.10/site-packages/sklearn/impute/_iterative.py:825: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [19]:
print(non_mis_df)

        regio  gm_naam    year    woz   a_inw   a_man  a_vrouw      a_geb  \
0       264.0      0.0  2014.0  300.0  7885.0  3900.0   3980.0  65.000000   
1       264.0      0.0  2015.0  294.0  7955.0  3935.0   4020.0  60.000000   
2       264.0      0.0  2016.0  298.0  8055.0  3980.0   4075.0  65.000000   
3       264.0      0.0  2017.0  312.0  8145.0  4025.0   4120.0  60.000000   
4       264.0      0.0  2018.0  339.0  8210.0  4030.0   4175.0  65.000000   
...       ...      ...     ...    ...     ...     ...      ...        ...   
51849  1551.0    202.0  2020.0  466.0   515.0   260.0    260.0   0.000000   
51850  1551.0    202.0  2021.0  478.0   510.0   255.0    255.0   5.000000   
51851  1551.0    202.0  2022.0  534.0   525.0   265.0    260.0   0.000000   
51852  1551.0    202.0  2023.0  581.0   490.0   260.0    230.0   5.000000   
51853  1551.0    202.0  2024.0  599.0   495.0   265.0    230.0   7.953722   

           p_geb       a_ste  ...  g_afs_kv  g_afs_sc  g_3km_sc  a_opp_ha  

In [20]:
#print(non_mis_df.isnull().sum())

end_df = non_mis_df.copy()

In [22]:
columns = end_df.columns.drop(['gm_naam', 'regio'])

before_mice_df.loc[:, columns] = end_df.loc[:, columns]

print(before_mice_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51854 entries, 0 to 51853
Data columns (total 71 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   regio     51854 non-null  object 
 1   gm_naam   51854 non-null  object 
 2   gwb_code  51854 non-null  object 
 3   ind_wbi   51854 non-null  int64  
 4   year      51854 non-null  int64  
 5   woz       51854 non-null  float64
 6   a_inw     51854 non-null  float64
 7   a_man     51854 non-null  float64
 8   a_vrouw   51854 non-null  float64
 9   a_geb     51854 non-null  float64
 10  p_geb     51854 non-null  float64
 11  a_ste     51854 non-null  float64
 12  p_ste     51854 non-null  float64
 13  a_hh      51854 non-null  float64
 14  g_hhgro   51854 non-null  float64
 15  bev_dich  51854 non-null  float64
 16  a_woning  51854 non-null  float64
 17  p_1gezw   51854 non-null  float64
 18  p_mgezw   51854 non-null  float64
 19  p_leegsw  51854 non-null  float64
 20  p_koopw   51854 non-null  fl

/tmp/ipykernel_3932/966619918.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[7885. 7955. 8055. ...  525.  490.  495.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  before_mice_df.loc[:, columns] = end_df.loc[:, columns]
/tmp/ipykernel_3932/966619918.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[3900. 3935. 3980. ...  265.  260.  265.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  before_mice_df.loc[:, columns] = end_df.loc[:, columns]
/tmp/ipykernel_3932/966619918.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[3980. 4020. 4075. ...  260.  230.  230.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  before_mice_df.loc[:, columns]

In [24]:
before_mice_df.head()

regio        gm_naam    gwb_code  ind_wbi  year    woz   a_inw  \
0  Belgisch Park  's-Gravenhage  BU05180271        1  2014  300.0  7885.0   
1  Belgisch Park  's-Gravenhage  BU05180271        1  2015  294.0  7955.0   
2  Belgisch Park  's-Gravenhage  BU05180271        1  2016  298.0  8055.0   
3  Belgisch Park  's-Gravenhage  BU05180271        1  2017  312.0  8145.0   
4  Belgisch Park  's-Gravenhage  BU05180271        1  2018  339.0  8210.0   

    a_man  a_vrouw  a_geb  ...  g_afs_kv  g_afs_sc  g_3km_sc  a_opp_ha  \
0  3900.0   3980.0   65.0  ...       0.4       0.5      12.2     101.0   
1  3935.0   4020.0   60.0  ...       0.4       0.5      12.1     101.0   
2  3980.0   4075.0   65.0  ...       0.4       0.5      10.4     102.0   
3  4025.0   4120.0   60.0  ...       0.4       0.5      10.4     102.0   
4  4030.0   4175.0   65.0  ...       0.4       0.5      10.4     106.0   

   a_lan_ha  a_wat_ha  pst_mvp  pst_dekp  ste_mvs  ste_oad  
0     101.0       0.0   2587.0         3        2   2363.0  
1     101.0       0.0   2587.0         2        2   2464.0  
2     102.0       0.0   2587.0         2        2   2469.0  
3     102.0       0.0   2587.0         2        2   2438.0  
4     106.0       0.0   2587.0         3        2   2446.0  

[5 rows x 71 columns]

In [25]:
before_mice_df.to_csv('/home/wouter/Documents/Scriptie/predicting/ENDCBSFILE.csv', index=False)